In [38]:
from fit import MF
from dataloader import PointCloudDataloader
import torch
from helpers import DQ,ScalerBase,LogitTransformer

In [39]:
#middle
model=MF.load_from_checkpoint("/beegfs/desy/user/schnakes/calochallenge/CaloChallenge/ibacllj4/checkpoints/epoch=334-w1p=0.00099.ckpt")
#big
modelbig=MF.load_from_checkpoint("/beegfs/desy/user/schnakes/calochallenge/CaloChallenge/lfbzcvva/checkpoints/epoch=183-w1p=0.00271.ckpt")

dl=PointCloudDataloader(name="middle",batch_size=1024)
#dl_big=PointCloudDataloader(name="big",batch_size=1024)

dl.setup("validation")
#dl_big.setup("validation")

Lightning automatically upgraded your loaded checkpoint from v1.9.4 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../beegfs/desy/user/schnakes/calochallenge/CaloChallenge/ibacllj4/checkpoints/epoch=334-w1p=0.00099.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.9.4 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../beegfs/desy/user/schnakes/calochallenge/CaloChallenge/lfbzcvva/checkpoints/epoch=183-w1p=0.00271.ckpt`


In [44]:
middle=[]
model.load_datamodule(dl)
voxel_shape=(45,16,9)
c=0
g=0
with torch.no_grad():
    for i in dl.train_dataloader():
        with torch.no_grad():
            cond=i[2]
            E=cond.clone().exp()
            cond[:]=10
            
            cond=torch.cat((cond.reshape(-1,1),(~i[1]).float().sum(1).reshape(-1,1)),dim=-1).float()
            x=model.sampleandscale(i[0],mask=i[1].bool(),cond=cond,scale=True)
            voxel_tensor=torch.zeros((len(i[0]),*voxel_shape)).clone()
            break

In [42]:
voxel_tensor

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 

In [25]:
middle=[]
model.load_datamodule(dl)
voxel_shape=(45,16,9)
c=0
g=0
Es=[]
with torch.no_grad():
    for i in dl.train_dataloader():
        with torch.no_grad():
            cond=i[2]
            E=cond.clone().exp()
            cond=torch.cat((cond.reshape(-1,1),(~i[1]).float().sum(1).reshape(-1,1)),dim=-1).float()
            x=model.sampleandscale(i[0],mask=i[1].bool(),cond=cond,scale=True)
            voxel_tensor=torch.zeros((len(i[0]),*voxel_shape)).clone()
            E_tensor=torch.zeros((len(i[0]))).clone()
            for _,k in enumerate(x):
                values=k[:,0]
            
                indices=k[:,1:]
                g+=len(k)
                torch.clamp(indices[:,0],0,voxel_shape[0]-1,out=indices[:,0])
                torch.clamp(indices[:,1],0,voxel_shape[1]-1,out=indices[:,1])
                torch.clamp(indices[:,2],0,voxel_shape[2]-1,out=indices[:,2])
            
                indices=indices[values>0]
                values=values[values>0]
                counts=indices.unique()[1].max()
                # print(indices,values)
                if counts>1:
                    c+=counts.sum()
                
                voxel_tensor[_]=torch.sparse_coo_tensor(indices=indices.T,values=values,size=voxel_shape).coalesce().to_dense()
            E_tensor=E.exp()
            middle.append(voxel_tensor)
            Es.append(E_tensor)  

/home/kaechben/CaloChallenge/helpers.py:337: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-X))
/home/kaechben/CaloChallenge/helpers.py:337: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-X))
/home/kaechben/CaloChallenge/helpers.py:337: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-X))
/home/kaechben/CaloChallenge/helpers.py:337: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-X))
/home/kaechben/CaloChallenge/helpers.py:337: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-X))
/home/kaechben/CaloChallenge/helpers.py:337: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-X))
/home/kaechben/CaloChallenge/helpers.py:337: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-X))
/home/kaechben/CaloChallenge/helpers.py:337: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-X))
/home/kaechben/CaloChallenge/helpers.py:337: Run

In [ ]:
indices.unique(dim=0,return_counts=True)[1].sum()

In [27]:
middle_=torch.vstack(middle).reshape(-1,voxel_shape[0]*voxel_shape[1]*voxel_shape[2]).numpy()
middle_.shape

(100000, 6480)

In [33]:
E=torch.hstack(Es)


In [36]:
import h5py
import numpy as np
with h5py.File("middle2.hdf5", "w-") as h5_file:
    
    N = len(middle_) # find the length of my dataset
    h5_file.create_dataset('incident_energies', shape=(N, ), dtype=np.float32, data=E.numpy())
    h5_file.create_dataset('showers', shape=(N,voxel_shape[0]*voxel_shape[1]*voxel_shape[2] ), dtype=np.float32, data=middle_)

In [37]:
E

tensor([171.2046, 410.2558, 398.3346,  ...,   3.1442,   3.2919,   2.7658],
       dtype=torch.float64)